# Optimizing analysis parameters $L$ and $\epsilon^2$
This notebooks shows different ways to optimize the analysis parameters.

In [1]:
using DIVAnd
using PyPlot
using Dates
using Statistics
include("../config.jl")

"https://dox.uliege.be/index.php/s/8tRk0NAStr2P70j/download"

## Data reading

In [2]:
varname = "Salinity"
filename = salinityprovencalfile
download_check(salinityprovencalfile, salinityprovencalfileURL)

obsval,obslon,obslat,obsdepth,obstime,obsid = loadobs(Float64, filename, varname);

[ Info: File already downloaded


### Topography and grid definition

See the notebook on [bathymetry](../2-Preprocessing/06-topography.ipynb) for more explanations.

In [ ]:
dx = dy = 0.125/2.
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(lonr, latr)

bathname = gebco04file
download_check(gebco04file, gebco04fileURL)

Reading the bathymetry and creating the mask

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr)

mask = falses(size(b,1),size(b,2))

for j = 1:size(b,2)
    for i = 1:size(b,1)
        mask[i,j] = b[i,j] >=1.0
    end
end

## Data selection for example

Cross validation, error calculations etc. assume independant data. Hence do not take high-resolution vertical profiles with all data but restrict yourself to specific small depth ranges. Here August data near the surface surface.     
We also perform a test on values to eliminate obvious outliers:

In [ ]:
datadepth=1
depthprecision=0.5

sel = (obsdepth .< datadepth) .& 
(obsdepth .>= (datadepth-depthprecision)) .& 
(Dates.month.(obstime) .== 8) .& 
(obsval .> 37)

obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];
@show (size(obsval))
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

We modify data weight by taking into account close points.      
**⚠️** This operation is particularly costly when dealing with big datasets.

In [ ]:
?DIVAnd.weight_RtimesOne

In [ ]:
rdiag=1.0./DIVAnd.weight_RtimesOne((obslon,obslat),(0.03,0.03))
@show maximum(rdiag),mean(rdiag)

## Analysis

Analysis `fi` using mean data as background.      
Structure `s` is stored for later use.

In [ ]:
len=1
epsilon2=1
fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2*rdiag);

Generate some plots:
1. Analysis with data points
2. Data residuals
3. Residuals vs value

In [ ]:
figure()
pcolor(xi,yi,fi/+mean(obsval),vmin=37,vmax=38.5);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
dataresiduals=DIVAnd_residualobs(s,fi)
scatter(obslon,obslat,s=2,c=obsval,vmin=37,vmax=38.5)
aspectratio = 1/cos(mean([ylim()...]) * pi/180)
gca().set_aspect(aspectratio)
rscale=sqrt(var(obsval))

In [ ]:
figure()
scatter(obslon,obslat,s=2,c=dataresiduals,vmin=-rscale,vmax=rscale,cmap=ColorMap("RdBu_r"));
colorbar(orientation="horizontal")
gca().set_aspect(aspectratio)
title("Residuals");

In [ ]:
figure()
scatter(obsval, dataresiduals, s=2)
title("Residuals as function of value");
xlabel("Salinity")
ylabel("Residuals");

# Cross validation

Take out data and measure difference between these data points not used and the analysis. Three methods are implemented 
## Define method used
    # 1: full CV
    # 2: sampled CV
    # 3: GCV
    # 0: automatic choice between the three possible ones, default value

In [ ]:
bestfactorl=ones(4)
bestfactore=ones(4)
for imeth=0:3

    bestfactorl[imeth+1],bestfactore[imeth+1], cvval,cvvalues, x2Ddata,y2Ddata,cvinter,xi2D,yi2D = DIVAnd_cv(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2*rdiag,2,3,imeth);
    @show bestfactorl[imeth+1],bestfactore[imeth+1]
    
    subplot(2,2,imeth+1)
    pcolor(xi2D,yi2D,cvinter)#,vmin=0,vmax=0.04)
    colorbar()
    xlabel("Log10 scale factor L")
    ylabel("Log10 scale factor e2")
    plot(x2Ddata,y2Ddata,".")
    plot(log10.(bestfactorl[imeth+1]), log10.(bestfactore[imeth+1]),"o")
    title("Method $imeth")
end

**⚠️ WARNING:** any tests with resulting length scales being smaller than around 4 times the grid spacing are meaningless.

Analysis with optimized values:


In [ ]:
newl=len*bestfactorl[2]
newe=epsilon2*bestfactore[2]

fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),newl,newe*rdiag);

In [ ]:
"New L $(round(newl, digits=3)) and new e2 $(round(newe, digits=3))"

In [ ]:
pcolor(xi,yi,fi.+mean(obsval),vmin=37,vmax=38.5);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
dataresiduals=DIVAnd_residualobs(s,fi)
scatter(obslon,obslat,s=2,c=obsval,vmin=37,vmax=38.5)
title("New L  = $(round(newl, digits=3))\n new e2 = $(round(newe, digits=3))")
rscale=sqrt(var(obsval))

In [ ]:
figure()
scatter(obslon,obslat,s=2,c=dataresiduals,vmin=-rscale,vmax=rscale);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
title("Residuals")

In [ ]:
figure()
scatter(obsval,dataresiduals)
title("Residuals as function of value");

## Only one parameter optimized

If $L$ is fixed by other calibration, you can decide to optimize only $\epsilon^2$:

In [ ]:
lenfixed=1
epsilon2=.1
for imeth=0:3
    bestfactore[imeth+1], cvval,cvvalues, x2Ddata,cvinter,xi2D = 
    DIVAnd_cv(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),lenfixed,epsilon2*rdiag,0,4,imeth);

    subplot(2,2,imeth+1)
    plot(xi2D,cvinter,"-")
    xlabel("Log10 scale factor e2")
    plot(x2Ddata,cvvalues,".")
    plot(log10.(bestfactore[imeth+1]), cvval,"o")
    title("Method $imeth")
end

In [ ]:
newl=lenfixed
newe=epsilon2*bestfactore[3]
@show newe
fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),newl,newe*rdiag);

In [ ]:
figure()
pcolor(xi,yi,fi.+mean(obsval),vmin=37,vmax=38.5);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
dataresiduals=DIVAnd_residualobs(s,fi)
scatter(obslon,obslat,s=2,c=obsval,vmin=37,vmax=38.5)

rscale=sqrt(var(obsval))

In [ ]:
figure()
scatter(obslon,obslat,s=2,c=dataresiduals,vmin=-rscale,vmax=rscale);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
title("Residuals")

In [ ]:
figure()
scatter(obsval,dataresiduals)
title("Residuals as function of value");

## Adaptive method

> G. Desroziers, L. Berre, B. Chapnik and P. Poli (2005)       
Diagnosis of observation, background and analysis-error statistics in observation space      
_Q. J. R. Meteorol. Soc._, __131__, pp. 3385–3396      
> doi: [10.1256/qj.05.108](https://rmets.onlinelibrary.wiley.com/doi/abs/10.1256/qj.05.108).

This adaptive method activated by call with 0,0 points to sample.

In [ ]:
myiterations=7
cvbest2=zeros(myiterations);
eps2=zeros(myiterations)
epsilon2=1
for i=1:myiterations
    cvval,factor=DIVAnd_cv(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),lenfixed,epsilon2*rdiag,0,0,3);
    eps2[i]=epsilon2;
    cvbest2[i]=cvval;
    epsilon2=epsilon2*factor
    @show epsilon2
end

Other direct application of Desroziers

In [ ]:
myiterations=7

eps2=zeros(myiterations)
epsilon2=1
for i=1:myiterations
    fit,sit=DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),lenfixed,epsilon2*rdiag);
    eps2[i]=epsilon2;
    factor=DIVAnd_adaptedeps2(sit,fit)
    epsilon2=epsilon2*factor
    @show epsilon2
end

Perform a new analysis with the optimized value of $\epsilon^2$:

In [ ]:
newl=lenfixed
newe=epsilon2
@show newe
fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),newl,newe*rdiag);

In [ ]:
figure()
pcolor(xi,yi,fi.+mean(obsval),vmin=37,vmax=38.5);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
dataresiduals=DIVAnd_residualobs(s,fi)
scatter(obslon,obslat,s=2,c=obsval,vmin=37,vmax=38.5)

rscale=sqrt(var(obsval))

In [ ]:
figure()
scatter(obslon,obslat,s=2,c=dataresiduals,vmin=-rscale,vmax=rscale);
colorbar(orientation="horizontal")
gca().set_aspect(1/cos(mean([ylim()...]) * pi/180))
title("Residuals")

In [ ]:
figure()
scatter(obsval,dataresiduals)
title("Residuals as function of value");

In [ ]:
var(dataresiduals),var(obsval),var(fi[(fi.!==NaN)])

## More information

using DIVAnd

In [3]:
?DIVAnd_cv

search: DIVAnd_cv DIVAnd_qc DIVAnd_cpme DIVAnd_save DIVAndrun DIVAndgo DIVAndfun



```
bestfactorl,bestfactore, cvval,cvvalues, x2Ddata,y2Ddata,cvinter,xi2D,yi2D = DIVAnd_cv(mask,pmn,xi,x,f,len,epsilon2,nl,ne,method;...);
```

Performs a cross validation to estimate the analysis parameters (correlation length and signal-to-noise ratio).

# Input

Same as for `DIVAndrun` with three more parameters `nl`,`ne` and `method`

  * `mask`: binary mask delimiting the domain. true is inside and false outside.

For oceanographic application, this is the land-sea mask.

  * `pmn`: scale factor of the grid. pmn is a tuple with n elements. Every      element represents the scale factor of the corresponding dimension. Its      inverse is the local resolution of the grid in a particular dimension.
  * `xi`: tuple with n elements. Every element represents a coordinate of the final grid on which the observations are interpolated
  * `x`: tuple with n elements. Every element represents a coordinate of the observations
  * `f`: value of the observations *minus* the background estimate (m-by-1 array).   (see note)
  * `len`: correlation length
  * `epsilon2`: error variance of the observations (normalized by the error variance of the background field). `epsilon2` can be a scalar (all observations have the same error variance and their errors are decorrelated), a vector (all observations can have a difference error variance and their errors are decorrelated) or a matrix (all observations can have a difference error variance and their errors can be correlated). If `epsilon2` is a scalar, it is thus the *inverse of the signal-to-noise ratio*.
  * `nl`: number of testing points around the current value of L. `1` means one additional point on both sides of the current L. `0` is allowed and means the parameter is not optimised.
  * `ne`: number of testing points around the current value of epsilon2. `0` is allowed as for `nl`
  * `method`: cross validation estimator method 1: full CV 2: sampled CV 3: GCV 0: automatic choice between the three possible ones, default value
  * Optional input arguments specified via keyword arguments are the same as for `DIVAnd`

# Output:

  * `bestfactorl`: best estimate of the multiplication factor to apply to len
  * `bestfactore`: best estimate of the multiplication factor to apply to epsilon2
  * `cvvales` : the cross validation values calculated
  * `factors` : the tested multiplication factors
  * `cvinter` : interpolated cv values for final optimisation
  * `X2Data, Y2Data` : coordinates of sampled cross validation in `L,epsilon2` space . Normally only used for debugging or plotting
  * `Xi2D, Yi2D` : coordinates of interpolated estimator . Normally only used for debugging or plotting

The output `bestfactorl` and `bestfactore` represent multiplication factors which should be applied to `L` and `epsilon2`.

The `len` and `epsilon2` provided should be close the real one as the tests will be performed around.


=========================================================================================================================

# Exercise

1. Redo for different data by changing the datadepth parameter introduced in the data selection part.      
(Compare surface behaviour with deeper regions by using another `datadepth` value).
2. Take out the data weight modification.
3. Possibly force the cross-validation method (use `?DIVAnd_cv`).
4. Once opimized, try to redo optimization with starting point being the first estimate.

## ⚠️ Important note
Remember: optimization relies on a series of hypotheses.       
In particular, data independance and isotropy is very often NOT ensured.       
When in doubt, increase $\epsilon^2$ and/or check for "duplicates".